<a href="https://colab.research.google.com/github/fanTaux/GoogleCollabQC2025/blob/main/3.%20Penghitung%20Presensi%20Maba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
from google.colab import files # Import library untuk upload file

def baca_file_data(pesan_prompt, allow_multiple=False):
    """
    Fungsi serbaguna untuk membaca file .csv atau .xlsx dari hasil upload.
    Akan terus bertanya hingga file yang valid diupload.
    Jika allow_multiple=True, akan membaca semua file yang diupload dan menggabungkannya.
    """
    df = None
    while True:
        try:
            print(f"▶️ {pesan_prompt}")
            uploaded = files.upload() # Minta pengguna mengupload file

            if not uploaded:
                print("❌ Tidak ada file yang diupload. Mohon upload file yang sesuai.")
                continue

            list_df = []
            for file_path in uploaded.keys():
                # 1. Periksa apakah file ada di lokasi yang diberikan (setelah diupload, file ada di direktori saat ini)
                # Tidak perlu os.path.exists() karena file sudah diupload

                # 2. Baca file berdasarkan ekstensinya (.csv atau .xlsx)
                if file_path.lower().endswith('.csv'):
                    # Gunakan BytesIO untuk membaca dari memory, bukan path
                    from io import BytesIO
                    list_df.append(pd.read_csv(BytesIO(uploaded[file_path]), dtype={'NIM': str})) # Baca NIM sebagai teks/string
                    print(f"✅ Berhasil membaca file CSV: {file_path}")
                elif file_path.lower().endswith('.xlsx'):
                    # Gunakan BytesIO untuk membaca dari memory, bukan path
                    from io import BytesIO
                    list_df.append(pd.read_excel(BytesIO(uploaded[file_path]), dtype={'NIM': str})) # Baca NIM sebagai teks/string
                    print(f"✅ Berhasil membaca file Excel: {file_path}")
                else:
                    print(f"❌ Format file '{file_path}' tidak didukung. Harap gunakan file dengan ekstensi .csv atau .xlsx.")
                    # Hapus file yang tidak didukung setelah diupload
                    os.remove(file_path)
                    print(f"File {file_path} dihapus.")


            if list_df:
                df = pd.concat(list_df, ignore_index=True)
                break # Keluar dari loop while True jika ada file yang berhasil dibaca
            else:
                 print("❌ Tidak ada file yang valid diupload. Mohon upload file yang sesuai.")


        except Exception as e:
            print(f"❌ Gagal membaca file. Terjadi error: {e}")
            # Jika gagal, loop akan berlanjut dan meminta input lagi

    return df

def hitung_nilai_presensi(jumlah_hadir, threshold):
    """Menghitung nilai akhir berdasarkan kehadiran dan threshold."""
    if threshold <= 0: # Menghindari pembagian dengan nol atau threshold negatif
        return 0
    elif jumlah_hadir >= threshold:
        return 100
    else:
        return (jumlah_hadir / threshold) * 100

# --- Bagian Utama Program ---

# 1. Input Threshold dari Pengguna
while True:
    try:
        threshold_input = input("▶️ Masukkan threshold jumlah presensi (angka): ")
        THRESHOLD_PRESENSI = int(threshold_input)
        print(f"✅ Threshold diatur ke: {THRESHOLD_PRESENSI}\n")
        break
    except ValueError:
        print("❌ Input tidak valid. Harap masukkan angka.")

# 2. Upload File Data Mahasiswa Baru
print("--- Upload File Data Mahasiswa ---")
df_maba = baca_file_data("Mohon upload file data maba (.csv atau .xlsx) yang berisi kolom 'NIM' dan 'Nama'")

# 3. Upload File Data Presensi
print("\n--- Upload File Data Presensi ---")
df_presensi = baca_file_data("Mohon upload file data presensi (.csv atau .xlsx) yang berisi kolom 'NIM')", allow_multiple=True)


# 4. Proses Perhitungan Data
if df_maba is not None and df_presensi is not None:
    try:
        # Pastikan kolom 'NIM' ada di df_presensi sebelum pemrosesan
        if 'NIM' not in df_presensi.columns:
             raise KeyError("Kolom 'NIM' tidak ditemukan di file presensi.")

        # Membersihkan NIM dari spasi dan tanda kutip di df_presensi
        df_presensi['NIM'] = df_presensi['NIM'].astype(str).str.strip().str.replace("'", "")

        # Hitung frekuensi kehadiran setiap NIM dari file presensi
        jumlah_kehadiran = df_presensi['NIM'].value_counts().reset_index()
        jumlah_kehadiran.columns = ['NIM', 'Jumlah Hadir']

        # Gabungkan data maba dengan data kehadiran
        # 'how=left' memastikan semua maba tetap ada di daftar akhir
        # Pastikan kolom 'NIM' ada di df_maba sebelum merge
        if 'NIM' not in df_maba.columns:
             raise KeyError("Kolom 'NIM' tidak ditemukan di file data maba.")

        # Membersihkan NIM dari spasi dan tanda kutip di df_maba (untuk konsistensi)
        df_maba['NIM'] = df_maba['NIM'].astype(str).str.strip().str.replace("'", "")


        df_hasil = pd.merge(df_maba, jumlah_kehadiran, on='NIM', how='left')

        # Ganti nilai NaN (tidak ada data presensi) dengan 0
        df_hasil['Jumlah Hadir'] = df_hasil['Jumlah Hadir'].fillna(0).astype(int)

        # Hitung nilai presensi untuk setiap baris
        df_hasil['Nilai Presensi'] = df_hasil['Jumlah Hadir'].apply(
            lambda hadir: hitung_nilai_presensi(hadir, THRESHOLD_PRESENSI)
        )
        df_hasil['Nilai Presensi'] = df_hasil['Nilai Presensi'].round(2)

        # 5. Tampilkan dan Simpan Hasil
        print("\n--- Hasil Akhir Perhitungan Presensi ---")
        print(df_hasil)

        FILE_HASIL_OUTPUT = 'hasil_presensi_final.xlsx'
        df_hasil.to_excel(FILE_HASIL_OUTPUT, index=False)
        print(f"\n✅ Perhitungan selesai. Hasil telah disimpan ke file: {FILE_HASIL_OUTPUT}")

    except KeyError as e:
        print(f"\n❌ ERROR: Kolom {e} tidak ditemukan di salah satu file. Pastikan nama kolom 'NIM' sudah benar dan sama di kedua file.")
    except Exception as e:
        print(f"\n❌ Terjadi error saat memproses data: {e}")